In [70]:
import pandas as pd
import numpy as np
data_path = "July2021_for_test.xlsx"
df = pd.read_excel(data_path, usecols="A:Q", engine="openpyxl")
print(df["Плавок(окр)"].sum())

900


In [71]:
index = np.where (df["Плавок(окр)"] >=2)
for i in index[0]:
    count = df.iloc[i]["Плавок(окр)"]
    #print(count)
    #remain = df.at[i, "Плавок(окр)"]%18
    df.at[i, "Плавок(окр)"] = 1
    count -=1
    while(count>0):
        df1 = df.loc[[i],:]
        df1["Плавок(окр)"] = 1
        df = pd.concat([df, df1], ignore_index=True)
        count-=1
        
#    df1 = df.loc[[i],:]
#    df1["Плавок(окр)"] = remain
#    df = pd.concat([df, df1], ignore_index=True)

In [72]:
df["imax"] = df["Диапазон изменения d от 1"]*16

In [73]:
df = df.sort_values(by=["Диапазон изменения d от 1"]).reset_index()

In [74]:
df.loc[df["imax"] ==80, "imax"]=32
df.loc[df["imax"] ==240, "imax"]=80

In [75]:
df[df["imax"]>80]

,index,Номер(i),Толщ.,Шир.,Диапазон изменения d от 1,ПОРТ,АПК,Плавок(окр),Разрядов(L_i),У6,У7,У8,У9 Fix,У9,Подходит УНРС,Диапазон изменения k,Число переменных,Номер толщины(H_i),imax
18,771,106,250,2080,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
19,78,78,250,1850,9,Новороссийск,1,1,0,1,1,1,0,1,4,"0,1,2,3",0,1,144
20,102,102,250,2075,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
21,768,106,250,2080,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
22,747,102,250,2075,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
23,748,102,250,2075,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
24,749,102,250,2075,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
25,750,102,250,2075,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
26,769,106,250,2080,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
27,106,106,250,2080,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144


In [76]:
# Array for thicknesses by number of heat
w_numbers = df["Номер толщины(H_i)"].to_numpy()
# Array for imax
j_imax = df["imax"].to_numpy() 

In [77]:
from amplify import (
    BinaryPoly,
    BinaryQuadraticModel,
    sum_poly,pair_sum,
    gen_symbols,
    Solver,
    decode_solution,
)
from amplify import Solver
from amplify.client import FixstarsClient
#from amplify.client.ocean import DWaveSamplerClient
from amplify.constraint import less_equal, equal_to, penalty,one_hot

def flatten(t):
    return [item for sublist in t for item in sublist]

client = FixstarsClient()
client.token = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
client.parameters.timeout = 60000  # Timeout is 60 seconds

solver = Solver(client)

In [100]:
#n = 80 #Number of timeslots for 5 days
n = 10
#m = 795 #Number of heats
m = 100
w = 6 #Number of thicknesses
w_max = [3,4,0,1,1,1] #Max thicknesses per UNRS

x = gen_symbols(BinaryPoly,m,n) #Variable x - heat for timeslot

In [101]:
# Condition for min(n, imax[j])
for j in range (m):
    if (df.loc[j, "imax"]<n):
        for i in range (df.loc[j, "imax"], n):
            x[j][i]=0

In [102]:
# В одном слоте 4 плавки по кол-ву УНРС
# Либо для 5.1. Constraints on columns
A = 1 #Coeff
col_constraints = [
    equal_to(sum_poly([x[j][i] for j in range(m)]), 4) for i in range(n)
]

In [103]:
H1_constr = A*sum(col_constraints)

In [104]:
# 5.2 Deadline
S = [int(df.loc[j,"imax"]<=n) for j in range(m)] #плавка должна быть разлита обязательно - взять из df
B=1
R = 1

In [105]:
H2_0 = sum_poly(m, lambda j: S[j]*(1 - sum_poly(n, lambda i:(x[j][i])))**2)

In [106]:
# Либо для 5.2 H2_1 Constraints on rows
row_constraints = 0
for j in range(m):
    if (1-S[j]):
        row_constraints += less_equal(sum_poly([x[j][i] for i in range(n)]), 1)

H2_1_constr = sum(row_constraints)

In [107]:
#H2 = B*H2_0 + R*H2_1_constr

In [108]:
#H2.num_logical_vars

In [109]:
# 5.3 Ограничение на количество плавок одной толщины в каждом ts
w_sum = [[0 for x in range(w)] for y in range(n)] #array for sums of each thickness for each ts
#constr_w = 0
# Number of heats for each thickness for each ts
for i in range(n):
    for w_ind in range(w):
        for j in range(m):
            w_sum[i][w_ind]+= (int(w_numbers[j]==w_ind))*x[j][i]
        #constr_w += less_equal(w_sum[i][w_ind], w_max[w_ind], label=label)       

In [112]:
#Create array x_w_full for slack variables
x_w_full = []
x_w_start = n*m
#for i in range(n):
for w_ind in range(w):
    w_max_ind = w_max[w_ind]+1
    x_w_full.append(gen_symbols(BinaryPoly, x_w_start, (w_max_ind,n)))
    x_w_start += w_max_ind*n       

In [114]:
H3 = 0
x_w_start = n*m
for i in range (n):
    for w_ind in range(w):
        #print(w_ind, w_sum[i][w_ind])
        H3_i_w = 0
        H3_i_w += w_sum[i][w_ind]
        #print(H3_i_w)
        for ww in range (1, w_max[w_ind]+1):
            x_w_start += ww
            H3_i_w +=ww*x_w_full[w_ind][ww][i]
            #print(H3_i_w)
        x_w_start += w_max[w_ind]*n    
        H3_i_w = (H3_i_w - w_max[w_ind])**2 
        H3+=H3_i_w   

In [ ]:
# H4 APK(L) ACV(V)
"""
K = 1 #Coeff
L = []
V = [] # TBD
H4 = 0
for j in range (m):
    L.append(df.loc[j,"АПК"])   
for i in range(n):
    H4 += less_equal(sum_poly(m, lambda j:(L[j]*x[j][i])), 2)
H4 = K*sum(H4)  
"""

In [ ]:
# 5.5 Логистические расходы
"""
F = 1 #Coeff
H5 = F*(sum_poly(m, lambda j: (j_imax[j] - sum_poly(n, lambda i:(i*x[j][i])))**2))
"""

In [115]:
# Array for offset of slack variables (distance - for count numbers of heats with different thicknesses) 
D_w = [0 for x in range(w)]
for ww in range(w):
    D_w[ww] = [[abs(y-x) for x in range(w_max[ww]+1)] for y in range(w_max[ww]+1)]

In [131]:
# 5.5 Перенастройки по толщине
# Для каждой толщины считаем, сколько плавок данной толщины делалось в соседних ts. Минимизируем дельту
# We can use array w_sum = [[0 for x in range(w)] for y in range(n)]

Q1 = 50 #Coeff   
Q2 = 10 #Coeff

H_5_i_0 = 0
for w_ind in range(w):
    #print("w_ind", w_ind)
    for i in range(n):
        H_5_i_w = 0
        #H_5_i_w +=(1-sum_poly(w_max[w_ind]+1,lambda ww: (x_w_full[w_ind][ww][i])))**2 
        H_5_i_w +=one_hot(sum_poly(w_max[w_ind]+1,lambda ww: (x_w_full[w_ind][ww][i])))
    H_5_i_0 += H_5_i_w
    
H_5_i_1 = 0    
for i in range(n-1):
    for w_ind in range(w):
        H_5_i_1_w = 0 #for each w
        for e_ind in range(w_max[w_ind]+1):
            for f_ind in range(w_max[w_ind]+1):
                H_5_i_1_w+= 0.5*D_w[w_ind][e_ind][f_ind]*x_w_full[w_ind][e_ind][i]*x_w_full[w_ind][f_ind][i+1]  

In [132]:
#H = H5 + H2_0 + H1_constr + H2_1_constr + H3 + H4 + H6

In [133]:
model = Q2*H_5_i_1 + H3 + H2_0 + H1_constr + H2_1_constr + Q1*H_5_i_0
#model = H2_0 + H1_constr + H2_1_constr

In [134]:
model.num_logical_vars

1106

In [135]:
count = 0
for count in range (1):
    result = solver.solve(model)
    if len(result) == 0:
        raise RuntimeError("Any one of constraints is not satisfied.")

    energy, values = result[0].energy, result[0].values

    x_values = decode_solution(x, values)

    print(energy)
    
    x_values_array = np.asarray(x_values, dtype = np.int32)
    np.savetxt("slabs-"+str(pd.Timestamp.now())+".csv", x_values_array, fmt='%i', delimiter=",")
    
    count+=1

0.0


In [136]:
for i in range (n):
    print("Timeslot ", i)
    for j in range(m):
        if (x_values[j][i]):
            print("Heat ", j)
            print ("Thickness ", w_numbers[j])
            print (x_values[j][i])
    
    

Timeslot  0
Heat  1
Thickness  3
1.0
Heat  41
Thickness  1
1.0
Heat  62
Thickness  1
1.0
Heat  85
Thickness  1
1.0
Timeslot  1
Heat  39
Thickness  1
1.0
Heat  40
Thickness  1
1.0
Heat  46
Thickness  1
1.0
Heat  52
Thickness  1
1.0
Timeslot  2
Heat  53
Thickness  1
1.0
Heat  84
Thickness  1
1.0
Heat  95
Thickness  1
1.0
Heat  99
Thickness  1
1.0
Timeslot  3
Heat  0
Thickness  3
1.0
Heat  22
Thickness  1
1.0
Heat  56
Thickness  1
1.0
Heat  86
Thickness  1
1.0
Timeslot  4
Heat  7
Thickness  3
1.0
Heat  36
Thickness  1
1.0
Heat  67
Thickness  1
1.0
Heat  91
Thickness  1
1.0
Timeslot  5
Heat  21
Thickness  1
1.0
Heat  43
Thickness  1
1.0
Heat  54
Thickness  1
1.0
Heat  63
Thickness  1
1.0
Timeslot  6
Heat  10
Thickness  3
1.0
Heat  68
Thickness  1
1.0
Heat  80
Thickness  1
1.0
Heat  90
Thickness  1
1.0
Timeslot  7
Heat  14
Thickness  3
1.0
Heat  70
Thickness  1
1.0
Heat  71
Thickness  1
1.0
Heat  98
Thickness  1
1.0
Timeslot  8
Heat  12
Thickness  3
1.0
Heat  42
Thickness  1
1.0
Heat  47
Th

In [137]:
client_result = solver.client_result
client_result.annealing_time_ms

59972

In [138]:
client_result.timing.cpu_time

56.50865599999748

In [139]:
def get_assignments(x_ij, rng=n):
    casts_assign = [[] for y in range(rng)]
    cast = 0
    for c in x_ij:
        if 1 in c:
            casts_assign[list(c).index(1)].append(cast)
        cast += 1
    return casts_assign

def get_thickness_by_cast_num(c):
    return df["Номер толщины(H_i)"][c]

#xij = np.loadtxt(open('/home/eugene/__Work/Slabs/Code/slabs-2021-11-09 00 01 03.398912.csv', "r"), delimiter=",")  # or just take the decoded solution 2D-array

casts_assign = get_assignments(x_values)
print(casts_assign)

thickness_assign = [list(map(get_thickness_by_cast_num, slot)) for slot in casts_assign]
print(thickness_assign)

[[1, 41, 62, 85], [39, 40, 46, 52], [53, 84, 95, 99], [0, 22, 56, 86], [7, 36, 67, 91], [21, 43, 54, 63], [10, 68, 80, 90], [14, 70, 71, 98], [12, 42, 47, 66], [6, 16, 19, 25]]
[[3, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [3, 1, 1, 1], [3, 1, 1, 1], [1, 1, 1, 1], [3, 1, 1, 1], [3, 1, 1, 1], [3, 1, 1, 1], [3, 4, 1, 1]]


In [ ]:
def check_thickness_assign(d):
    if d[0] > 3:
        print('Thickness 0 casts number violation')
    if d[1] > 4:
        print('Thickness 1 casts number violation')
    if d[2] > 0:
        print('Thickness 2 casts number violation')
    if d[3] > 1:
        print('Thickness 3 casts number violation')
    if d[4] > 1:
        print('Thickness 4 casts number violation')
    if d[5] > 1:
        print('Thickness 5 casts number violation')
    else:
        print("Ok")

slot_num = 0
for slot in thickness_assign:
    d = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
    for thickness in slot:
        d[thickness] += 1
    print(str(d) + 'in slot ' + str(slot_num))
    check_thickness_assign(d)

    slot_num += 1

In [140]:
# Calculation for UNRS changeovers
slot_num = 0
d_array = []
d_array_diff = []
for slot in thickness_assign:
    d = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
    for thickness in slot:
        d[thickness] += 1
    d_array.append(d)
    slot_num += 1
    
count = 0   # number of changeovers 
for i in range(n-1):
    slot_a = d_array[i]
    slot_b = d_array[i+1]
    diff = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
    if(slot_a != slot_b):
        print("Change, slots:", i,"and ", i+1)
        for thickness in slot_b:
            if(slot_b[thickness] - slot_a[thickness]!=0):
                diff[thickness] = slot_b[thickness] - slot_a[thickness]
                count +=0.5 
        print(diff)    
print(count) 

Change, slots: 0 and  1
{0: 0, 1: 1, 2: 0, 3: -1, 4: 0, 5: 0}
Change, slots: 2 and  3
{0: 0, 1: -1, 2: 0, 3: 1, 4: 0, 5: 0}
Change, slots: 4 and  5
{0: 0, 1: 1, 2: 0, 3: -1, 4: 0, 5: 0}
Change, slots: 5 and  6
{0: 0, 1: -1, 2: 0, 3: 1, 4: 0, 5: 0}
Change, slots: 8 and  9
{0: 0, 1: -1, 2: 0, 3: 0, 4: 1, 5: 0}
5.0
